# Week 14: CNN Lab - Rock, Paper, Scissors

**Objective:** Build, train, and test a Convolutional Neural Network (CNN) to classify images of hands playing Rock, Paper, or Scissors.

### Step 1: Setup and Data Download

This first cell downloads the dataset from Kaggle.

In [1]:
import kagglehub

path = kagglehub.dataset_download("drgfreeman/rockpaperscissors")

print("Path to dataset files:", path)

c:\ML_F_PES2UG23CS350_Mohit Kumar\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Mohit\.cache\kagglehub\datasets\drgfreeman\rockpaperscissors\versions\2


In [2]:
import shutil
import os

# Try to use the path from the previous cell, otherwise fall back to common locations.
# We will detect the first folder that contains the class subfolders.
dst_root = "/content/dataset"
os.makedirs(dst_root, exist_ok=True)

folders_to_copy = ["rock", "paper", "scissors"]

# Build candidate roots that might contain class folders
candidates = []
# 'path' should come from the previous cell; if not defined, ignore it
if "path" in globals() and isinstance(path, str):
    candidates.append(path)
    candidates.append(os.path.join(path, "rockpaperscissors"))
    candidates.append(os.path.join(path, "Rock-Paper-Scissors"))
# Kaggle Notebook default
candidates.append("/kaggle/input/rockpaperscissors")
# Some users may manually unzip
candidates.append("/content/rockpaperscissors")
candidates.append("/content/Rock-Paper-Scissors")

src_root = None
for cand in candidates:
    if all(os.path.exists(os.path.join(cand, cls)) for cls in folders_to_copy):
        src_root = cand
        break

if src_root is None:
    raise FileNotFoundError(
        f"Could not find the dataset containing {folders_to_copy} in any known location. "
        f"Tried: {candidates}"
    )

print("Using dataset source:", src_root)

for folder in folders_to_copy:
    src_path = os.path.join(src_root, folder)
    dst_path = os.path.join(dst_root, folder)

    if os.path.exists(src_path):
        shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
        print("Copied:", folder)
    else:
        print("Folder not found:", folder)

Using dataset source: C:\Users\Mohit\.cache\kagglehub\datasets\drgfreeman\rockpaperscissors\versions\2
Copied: rock
Copied: paper
Copied: scissors


### Step 2: Imports and Device Setup

Import the necessary libraries and check if a GPU is available.

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np

# Set the 'device' variable
# Use CUDA if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)

Using device: cpu


### Step 3: Data Loading and Preprocessing

Here we will define our image transformations, load the dataset, split it, and create DataLoaders.

In [4]:
DATA_DIR = "/content/dataset"

# Define image transforms:
# 1. Resize to 128x128
# 2. Convert to Tensor
# 3. Normalize with mean=0.5 and std=0.5 (for each channel)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

# Load dataset using ImageFolder
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)

class_names = full_dataset.classes
print("Classes:", class_names)

# Split dataset: 80% train, 20% test
total_len = len(full_dataset)
train_size = int(0.8 * total_len)  # 80%
test_size = total_len - train_size  # remainder

# Use a fixed seed for reproducibility
train_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create DataLoaders
batch_size = 32
pin_memory = torch.cuda.is_available()

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=pin_memory
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=pin_memory
)

print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Test images: {len(test_dataset)}")

Classes: ['paper', 'rock', 'scissors']
Total images: 2188
Training images: 1750
Test images: 438


### Step 4: Define the CNN Model

Fill in the `conv_block` and `fc_block` with the correct layers.

In [5]:
class RPS_CNN(nn.Module):
    def __init__(self):
        super(RPS_CNN, self).__init__()

        # Convolutional blocks:
        # 1. Conv2d(3->16), ReLU, MaxPool2d(2)
        # 2. Conv2d(16->32), ReLU, MaxPool2d(2)
        # 3. Conv2d(32->64), ReLU, MaxPool2d(2)
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )

        # After 3 MaxPool(2) layers: 128 -> 64 -> 32 -> 16
        # Flattened size = 64 * 16 * 16
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(256, 3),  # 3 classes
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x

# Initialize model, criterion, optimizer
model = RPS_CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

RPS_CNN(
  (conv_block): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=256, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=3, bias=True)
  )
)


### Step 5: Train the Model

Fill in the core training steps inside the loop.

In [6]:
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()  # training mode
    total_loss = 0.0

    for images, labels in train_loader:
        # Move data to device
        images, labels = images.to(device), labels.to(device)

        # 1. Clear gradients
        optimizer.zero_grad()

        # 2. Forward pass
        outputs = model(images)

        # 3. Compute loss
        loss = criterion(outputs, labels)

        # 4. Backward pass
        loss.backward()

        # 5. Update weights
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader) if len(train_loader) > 0 else 0.0
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss = {avg_loss:.4f}")

print("Training complete!")

Epoch 1/10, Loss = 0.7192
Epoch 2/10, Loss = 0.2184
Epoch 3/10, Loss = 0.1049
Epoch 4/10, Loss = 0.0537
Epoch 5/10, Loss = 0.0267
Epoch 6/10, Loss = 0.0128
Epoch 7/10, Loss = 0.0125
Epoch 8/10, Loss = 0.0041
Epoch 9/10, Loss = 0.0031
Epoch 10/10, Loss = 0.0042
Training complete!


### Step 6: Evaluate the Model

Test the model's accuracy on the unseen test set.

In [7]:
model.eval()  # evaluation mode
correct = 0
total = 0

# No gradient needed during evaluation
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # 1. Get raw outputs (logits)
        outputs = model(images)

        # 2. Predicted class (argmax over dim=1)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100.0 * correct / total if total > 0 else 0.0
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 97.95%


### Step 7: Test on a Single Image

Let's see how the model performs on one image.

In [8]:
import glob

def predict_image(model, img_path):
    model.eval()

    img = Image.open(img_path).convert("RGB")
    # Apply the same transforms and add batch dimension
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        # 1. Get raw outputs (logits)
        output = model(img)

        # 2. Get predicted class index
        _, pred = torch.max(output, 1)

    return class_names[pred.item()]

# Option 1: Use a known existing image in 'paper' (choose first file found)
paper_files = sorted(glob.glob(os.path.join("/content/dataset", "paper", "*")))
if len(paper_files) == 0:
    raise FileNotFoundError("No images found in /content/dataset/paper. Check dataset setup.")
test_img_path = paper_files[0]

prediction = predict_image(model, test_img_path)
print(f"Model prediction for {test_img_path}: {prediction}")

Model prediction for /content/dataset\paper\04l5I8TqdzF9WDMJ.png: paper


### Step 8: Play the Game!

This code is complete. If your model is trained, you can run this cell to have the model play against itself.

In [9]:
import random
import os

def pick_random_image(class_name):
    folder = f"/content/dataset/{class_name}"
    files = os.listdir(folder)
    img = random.choice(files)
    return os.path.join(folder, img)

def rps_winner(move1, move2):
    if move1 == move2:
        return "Draw"

    rules = {
        "rock": "scissors",
        "paper": "rock",
        "scissors": "paper"
    }

    if rules[move1] == move2:
        return f"Player 1 wins! {move1} beats {move2}"
    else:
        return f"Player 2 wins! {move2} beats {move1}"

# -----------------------------------------------------------
# 1. Choose any two random classes
# -----------------------------------------------------------
choices = ["rock", "paper", "scissors"]
c1 = random.choice(choices)
c2 = random.choice(choices)

img1_path = pick_random_image(c1)
img2_path = pick_random_image(c2)

print("Randomly selected images:")
print("Image 1:", img1_path)
print("Image 2:", img2_path)

# -----------------------------------------------------------
# 2. Predict their labels using the model
# -----------------------------------------------------------
p1 = predict_image(model, img1_path)
p2 = predict_image(model, img2_path)

print("\nPlayer 1 shows:", p1)
print("Player 2 shows:", p2)

# -----------------------------------------------------------
# 3. Decide the winner
# -----------------------------------------------------------
print("\nRESULT:", rps_winner(p1, p2))

Randomly selected images:
Image 1: /content/dataset/rock\yQIWMi6PpMKRMZaa.png
Image 2: /content/dataset/paper\bxn0wMR4QebqVl62.png

Player 1 shows: rock
Player 2 shows: paper

RESULT: Player 2 wins! paper beats rock
